<h1> ch7 RNN을 사용한 문장 생성 </h1>

<p>
    이번 장에서는 언어 모델을 사용하여 '문장 생성'을 수행한다.
</p>

<h2> 7.1 언어 모델을 사용한 문장 생성 </h2>

<h3> 7.1.1 RNN을 사용한 문장 생성의 순서 </h3>

<p>
    학습된 Time LSTM 모델에 "I"라는 단어를 입력을 주면 단어의 확률분포를 출력할 것이다. 확률분포를 가지고 단어를 생성할 수 있다. 확률이 제일 높은 단어를 선택하거나, 확률을 기반으로 샘플링하여 단어를 생성할 수 있다. "I"라는 단어를 이용하여 "say"가 출력 되었다고 할 수 있다. 그렇다면 다시 "say"를 이용하여 반복해서 단어를 출력할 수 있다. 원하는 만큼 반복하거나, 특수한 구문이 출력하기 전까지(<eos>) 반복하여 문장을 생성시킬 수 있다.
</p>

<h3> 7.1.2 문장 생성 구현 </h3>

In [1]:
import sys
sys.path.append("..")
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]
        
        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())
            
            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))
                
        return word_ids

In [3]:
import sys
sys.path.append('..')
from rnnlm_gen import RnnlmGen
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('../ch06/Rnnlm.pkl')

# 시작(start) 문자와 건너뜀(skip) 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '\n')
print(txt)

you widen payments
 auto tucker owns innovation programs if it can be supported by him
 one new 's appointment said he is n't unlikely 's mr. roman
 lloyd 's is mr. roman
 in short his promotion news industry already has become part of an unwelcome plan by bear and federal regulators
 golden nathan whose violin is support to report a big suit hopes that in the death of nl between senate areas 's battered seven years ago which he is n't clear that mr. wathen is later this year
 some businessmen are now long


<h2> 7.2 seq2seq </h2>

<h3> 7.2.1 seq2seq 의 원리 </h3>

<p>
    seq2seq를 Encoder-Decoder 모델이라고 한다. 문자 그대로 Encoder는 입력 데이터를 인코딩하고, Decoder는 인코딩된 데이터를 디코딩한다. seq2seq 의 Encoder와 Decoder가 "나는 고양이로소이다"라는 문장을 "I am a cat"으로 번역해보겠다. 처음에는 Encoder 가 "나는 고양이로소이다"라는 출발어 문장을 인코딩한다. 인코딩한 정보를 Decoder에 전달하고, Decoder가 도착어 문장을 생성한다. 이때 Encoder가 인코딩한 정보에는 번역에 필요한 정보가 응축되어 있다. Encoder와 Decoder가 협력하여 시계열 데이터를 다른 시계열 데이터로 변환한다. 
</p>
<p>
    Encoder는 RNN을 이용해 시계열 데이터를 h라는 은닉 상태 벡터로 변환한다. RNN으로 LSTM이나 단순한 RNN, GRU 등도 이용할 수 있다. 벡터 h는 LSTM 계층의 마지막 은닉 상태이다. 은닉 상태 h에 입력 문장을 번역하는데 필요한 정보가 인코딩된다. 은닉 상태 h는 고정 길이 벡터이다. 결국 임의 길이의 문장을 고정 길이 벡터로 변환해 준다. 
</p>
<p>
    Decoder는 앞 절의 신경망과 같은 구성이다. 하지만 처음 신경망에서 LSTM 계층이 벡터 h를 입력받는다. 
</p>

<h3> 7.2.2 시계열 데이터 변환용 장난감 문제 </h3>

<p>
    간단한 '+' 더하기 문제를 다루겠다. 머신러닝을 평가하고자 만든 간단한 문제를 'toy problem'이라고 한다. 
</p>

<h3> 7.2.3 가변 길이 시계열 데이터 </h3>

<p>
    '덧셈'을 문자의 리스트로 다루기로 했다. 덧셈 문장의 문자 수가 문제마다 다르다는 것이다. 덧셈 문제에서는 샘플마다 데이터의 시간 방향 크기가 다르다. 신경망 학습 시 '미니배치 처리'를 하려면 무언가 추가 노력이 필요하다. 가변 길이 시계열 데이터를 미니배치로 학습하기 위한 가장 단순한 방법은 패딩을 사용하는 것이다. 패딩은 원래 데이터에 의미 없는 데이터를 채워 모든 데이터의 길이를 균일하게 맞추는 기법이다. 
</p>

<h3> 7.2.4 덧셈 데이터셋 </h3>

In [1]:
import sys
sys.path.append('..')
from dataset import sequence

(x_train, t_train), (x_test, t_test) = \
    sequence.load_data('addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)
# (45000, 7) (45000, 5)
# (5000, 7) (5000, 5)

print(x_train[0])
print(t_train[0])

print(''.join([id_to_char[c] for c in x_train[0]]))
print(''.join([id_to_char[c] for c in t_train[0]]))

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)
[ 3  0  2  0  0 11  5]
[ 6  0 11  7  5]
71+118 
_189 


<h2> 7.3 seq2seq 구현 </h2>

<h3> 7.3.1 Encoder 클래스 </h3>

<p>
    Encoder 클래스처럼 문자열을 받아 벡터 h로 변환한다. Encoder 클래스는 Embedding 계층과 LSTM 계층으로 구성된다. Embedding 계층에는 문자를 문자 벡터로 변환된다. LSTM 계층은 시간 방향에는 은닉 벡터와 셀, 위 방향으로는 은닉 상태만 출력한다. 하지만 위 계층에는 없기 때문에 위 출력은 폐기된다. 마지막 h를 Decoder에 전달한다. 
</p>

In [2]:
class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
        
        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        self.hs = None
        
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        self.hs = hs
        return hs[:, -1, :]
    
    def backward(self, dh):
        dhs = np.zeros_like(self.hs)
        dhs[:, -1. :] = dh
        
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout

<h3> 7.3.2 Decoder 클래스 </h3>

<p>
    Encoder 클래스가 출력한 h를 받아 다른 문자열을 출력한다. 정답 데이터는 "_62" 이고 출력 데이터는 '6', '2', '', ''이 되도록 학습시킨다. 이번 문제는 '덧셈'이므로 확률적 '비결정성'을 배제하고 '결정적'인 답을 생성하고자 한다. 즉, '확률적'이 아닌 '결정적'으로 선택한다. 
</p>

In [4]:
class Decoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.affine = TimeAffine(affine_W, affine_b)
        
        self.params, self.grads = [], []
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, xs, h):
        self.lstm.set_state(h)
        
        out = self.embed.forward(xs)
        out = self.lstm.forward(out)
        score = self.affine.forward(out)
        return score
    
    def backward(self, dscore):
        dout = self.affine.backward(dscore)
        dout = self.lstm.backward(dout)
        dout = self.embed.backward(dout)
        dh = self.lstm.dh
        return dh
    
    def generate(self, h, start_id, sample_size):
        sampled = []
        sample_id = start_id
        self.lstm.set_state(h)
        
        for _ in range(sample_size):
            x = np.array(sample_id).reshape((1, 1))
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)
            
            sample_id = np.argmax(score.flatten())
            sampled.append(int(sample_id))
            
        return sampled

<h3> 7.3.3 Seq2seq 클래스

In [7]:
class Seq2seq(BaseModel):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        self.encoder = Encoder(V, D, H)
        self.decoder = Decoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()
        
        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads
        
    def forward(self, xs, ts):
        decoder_xs, decoder_ts = ts[:, :-1], ts[:, 1:]
        
        h = self.encoder.forward(xs)
        score = self.decoder.forward(decoer_xs, h)
        loss = self.softmax.forward(score, decoder_ts)
        
    def backward(self, dout=1):
        dout = self.softmax.backward(dout)
        dh = self.decoder.backward(dout)
        dout = self.encoder.backward(dh)
        return dout
    
    def generate(self, xs, start_id, sample_size):
        h = self.encoder.forward(xs)
        sampled = self.decoder.generate(h, start_id, sample_size)
        return sampled

NameError: name 'BaseModel' is not defined

<h3> 7.3.4 seq2seq 평가 </h3>

<p>
    <ol>
        <li> 학습 데이터에서 미니배치 선택</li>
        <li> 미니배치로부터 기울기를 계산</li>
        <li> 기울기를 사용하여 매개변수를 갱신 </li>
    </ol>
</p>

In [ ]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from seq2seq import Seq2seq
from peeky_seq2seq import PeekySeq2seq

# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
x_train, x_test = x_train[:, ::-1], x_test[:, ::-1] # 입력데이터 반전
char_to_id, id_to_char = sequence.get_vocab()

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# 모델 / 옵티마이저 / 트레이너 생성
#model = Seq2seq(vocab_size, wordvec_size, hidden_size)
model = PeekySeq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1, 
                batch_size=batch_size, max_grad=max_grad)
    
    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                   id_to_char, verbose)
    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('검증 정확도 %.3f%%' % (acc * 100))

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 2.57
| 에폭 1 |  반복 21 / 351 | 시간 0[s] | 손실 2.48
| 에폭 1 |  반복 41 / 351 | 시간 0[s] | 손실 2.20
| 에폭 1 |  반복 61 / 351 | 시간 1[s] | 손실 1.99
| 에폭 1 |  반복 81 / 351 | 시간 1[s] | 손실 1.89
| 에폭 1 |  반복 101 / 351 | 시간 1[s] | 손실 1.82
| 에폭 1 |  반복 121 / 351 | 시간 2[s] | 손실 1.82
| 에폭 1 |  반복 141 / 351 | 시간 2[s] | 손실 1.80
| 에폭 1 |  반복 161 / 351 | 시간 2[s] | 손실 1.79
| 에폭 1 |  반복 181 / 351 | 시간 3[s] | 손실 1.78
| 에폭 1 |  반복 201 / 351 | 시간 3[s] | 손실 1.77
| 에폭 1 |  반복 221 / 351 | 시간 4[s] | 손실 1.76
| 에폭 1 |  반복 241 / 351 | 시간 4[s] | 손실 1.76
| 에폭 1 |  반복 261 / 351 | 시간 4[s] | 손실 1.75
| 에폭 1 |  반복 281 / 351 | 시간 5[s] | 손실 1.74
| 에폭 1 |  반복 301 / 351 | 시간 5[s] | 손실 1.74
| 에폭 1 |  반복 321 / 351 | 시간 5[s] | 손실 1.73
| 에폭 1 |  반복 341 / 351 | 시간 6[s] | 손실 1.73
Q   58+77
T 162 
X 100 
---
Q 461+579
T 1139
X 1013
---
Q  48+285
T 666 
X 102 
---
Q   551+8
T 163 
X 100 
---
Q  55+763
T 422 
X 1023
---
Q 752+006
T 857 
X 1023
---
Q 292+167
T 1053
X 1023
---
Q 795+038
T 1427
X 1111
---
Q  838+6

Q   58+77
T 162 
X 161 
---
Q 461+579
T 1139
O 1139
---
Q  48+285
T 666 
X 657 
---
Q   551+8
T 163 
X 155 
---
Q  55+763
T 422 
O 422 
---
Q 752+006
T 857 
O 857 
---
Q 292+167
T 1053
O 1053
---
Q 795+038
T 1427
X 1438
---
Q  838+62
T 864 
O 864 
---
Q  39+341
T 236 
O 236 
---
검증 정확도 65.060%
| 에폭 9 |  반복 1 / 351 | 시간 0[s] | 손실 0.32
| 에폭 9 |  반복 21 / 351 | 시간 0[s] | 손실 0.31
| 에폭 9 |  반복 41 / 351 | 시간 0[s] | 손실 0.31
| 에폭 9 |  반복 61 / 351 | 시간 1[s] | 손실 0.31
| 에폭 9 |  반복 81 / 351 | 시간 1[s] | 손실 0.29
| 에폭 9 |  반복 101 / 351 | 시간 1[s] | 손실 0.29
| 에폭 9 |  반복 121 / 351 | 시간 2[s] | 손실 0.29
| 에폭 9 |  반복 141 / 351 | 시간 2[s] | 손실 0.27
| 에폭 9 |  반복 161 / 351 | 시간 3[s] | 손실 0.27
| 에폭 9 |  반복 181 / 351 | 시간 3[s] | 손실 0.26
| 에폭 9 |  반복 201 / 351 | 시간 3[s] | 손실 0.25
| 에폭 9 |  반복 221 / 351 | 시간 4[s] | 손실 0.25
| 에폭 9 |  반복 241 / 351 | 시간 4[s] | 손실 0.24
| 에폭 9 |  반복 261 / 351 | 시간 5[s] | 손실 0.24
| 에폭 9 |  반복 281 / 351 | 시간 5[s] | 손실 0.23
| 에폭 9 |  반복 301 / 351 | 시간 5[s] | 손실 0.22
| 에폭 9 |  반복 321 / 351 

| 에폭 16 |  반복 181 / 351 | 시간 3[s] | 손실 0.04
| 에폭 16 |  반복 201 / 351 | 시간 3[s] | 손실 0.04
| 에폭 16 |  반복 221 / 351 | 시간 4[s] | 손실 0.05
| 에폭 16 |  반복 241 / 351 | 시간 4[s] | 손실 0.06
| 에폭 16 |  반복 261 / 351 | 시간 5[s] | 손실 0.06
| 에폭 16 |  반복 281 / 351 | 시간 5[s] | 손실 0.06
| 에폭 16 |  반복 301 / 351 | 시간 5[s] | 손실 0.05
| 에폭 16 |  반복 321 / 351 | 시간 6[s] | 손실 0.04
| 에폭 16 |  반복 341 / 351 | 시간 6[s] | 손실 0.04
Q   58+77
T 162 
O 162 
---
Q 461+579
T 1139
O 1139
---
Q  48+285
T 666 
O 666 
---
Q   551+8
T 163 
O 163 
---
Q  55+763
T 422 
O 422 
---
Q 752+006
T 857 
O 857 
---
Q 292+167
T 1053
O 1053
---
Q 795+038
T 1427
O 1427
---
Q  838+62
T 864 
O 864 
---
Q  39+341
T 236 
O 236 
---
검증 정확도 95.100%
| 에폭 17 |  반복 1 / 351 | 시간 0[s] | 손실 0.05
| 에폭 17 |  반복 21 / 351 | 시간 0[s] | 손실 0.03
| 에폭 17 |  반복 41 / 351 | 시간 0[s] | 손실 0.03
| 에폭 17 |  반복 61 / 351 | 시간 1[s] | 손실 0.03
| 에폭 17 |  반복 81 / 351 | 시간 1[s] | 손실 0.03
| 에폭 17 |  반복 101 / 351 | 시간 2[s] | 손실 0.03
| 에폭 17 |  반복 121 / 351 | 시간 2[s] | 손실 0.02
| 에폭 17

<h2> 7.4 seq2seq 개선 </h2>

<h3> 7.4.1 입력 데이터 반전(Reverse) </h3>

<p>
    첫 번째 개선안은 입력 데이터의 순서를 반전시키는 것이다. 이 트릭을 사용하면 많은 경우 학습 진행이 빨라져서, 최종 정확도도 좋아진다고 한다. 직관적으로 기울기 전파가 원할해져서 학습의 진행이 빨라지고 정확도가 향상된다. 
</p>

<h3> 7.4.2 엿보기(Peeky) </h3>

<p>
    현재 Encoder에서 출력한 h는 Decoder의 최초 시각 LSTM에서만 사용된다. 이 출력 h를 Decoder의 다른 계층에도 전해주는 것이다. 
</p>

In [14]:
class PeekyDecoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(H + D, 4 * H) / np.sqrt(H + D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H + H, V) / np.sqrt(H + H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.affine = TimeAffine(affine_W, affine_b)
        
        self.params, self.grads = [], []
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
        self.cache = None
        
    def forward(self, xs, h):
        N, T = xs.shape
        N, H = h.shape
        
        self.lstm.set_state(h)
        
        out = self.embed.forward(xs)
        hs = np.repeat(h, T, axis=0).reshape(N, T, H)
        out = np.concatenate((hs, out), axis=2)
        
        out = self.lstm.forward(out)
        out = np.concatenate((hs, out), axis=2)
        
        score = self.affine.forward(out)
        self.cache = H
        return score